In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

# Input data files are available in the read-only "../input/" directory
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import string
import re
# Load Data
test_data=pd.read_csv("/kaggle/input/titanic/test.csv")
train_data=pd.read_csv("/kaggle/input/titanic/train.csv")
test_data.info()
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass  

# Process useless fields
## Cabin  
 - extract deck  
 - some cabins are grouped
 - even/odd = port/starboard https://www.encyclopedia-titanica.org/titanic-deckplans/c-deck.html
## Ticket  
 - extract prefix
 - some tickets cover many passengers     
## Name  
 - extract title  
 - extract surname

In [ ]:
X=train_data.dropna()
sns.countplot(x='Port', hue='Survived',data=X)

In [2]:
# Cabin
############################################################################################################################################
"""
- Count of cabin groupings
- Cabin grouping
- Deck
"""  

data = [train_data, test_data]

for dataset in data:
    dataset['Cabin_count'] = dataset['Cabin'].apply(lambda x: len(str(x).split()) if pd.notnull(x) else 0)
    dataset['Deck'] = dataset['Cabin'].apply(lambda s: s[0] if pd.notnull(s) else 'M')
    
   # dataset['Port'] =dataset['Cabin'].astype(str).str.extract(r'(\d+)')
   # dataset['Port']= pd.to_numeric(dataset['Port'], errors='coerce').astype('Int64')%2==0

    idx = dataset[dataset['Deck'] == 'T'].index
    dataset.loc[idx, 'Deck'] = 'A'
    dataset['Deck'] = dataset['Deck'].replace(['A', 'B', 'C'], 'ABC')
    dataset['Deck'] = dataset['Deck'].replace(['D', 'E'], 'DE')
    dataset['Deck'] = dataset['Deck'].replace(['F', 'G'], 'FG')

    dataset=dataset.drop(["Cabin"], axis=1)
"""
import re
deck = {"A": "A", "B": "B", "C": "C", "D": "D", "E": "E", "F": "F", "G": "G", "U": "U"}
data = [train_data, test_data]

for dataset in data:
    dataset['Cabin'] = dataset['Cabin'].fillna("U0")
    dataset['Deck'] = dataset['Cabin'].map(lambda x: re.compile("([a-zA-Z]+)").search(x).group())
    idx = dataset[dataset['Deck'] == 'T'].index
    dataset.loc[idx, 'Deck'] = 'A'
    dataset['Deck'] = dataset['Deck'].map(deck)
    dataset['Deck'] = dataset['Deck'].fillna("U")
    #dataset['Deck'] = dataset['Deck'].astype(int)
"""
# we can now drop the cabin feature
train_data = train_data.drop(['Cabin'], axis=1)
test_data = test_data.drop(['Cabin'], axis=1)


In [3]:
# Ticket
############################################################################################################################################
"""
- Prefix
- Count 
- common ticket groupings
"""

# remove dots and slashes
def get_prefix(ticket):
    lead = ticket.split(' ')[0][0]
    if lead.isalpha():
        return 'Prefixed' #ticket.split(' ')[0]
    else:
        return 'NoPrefix'

data = [train_data, test_data]

for dataset in data:
    dataset['Ticket'] = dataset['Ticket'].replace('LINE','line 0')
    dataset['Ticket'] = dataset['Ticket'].apply(lambda x: x.replace('.','').replace('/','').lower())
    dataset['Prefix'] = dataset['Ticket'].apply(lambda x: get_prefix(x))

    dataset['TicketNumber'] = dataset['Ticket'].apply(lambda x: int(x.split(' ')[-1])//1)
    dataset['TNlength'] = dataset['TicketNumber'].apply(lambda x : len(str(x)))
    dataset['LeadingDigit'] = dataset['TicketNumber'].apply(lambda x : int(str(x)[0]))
    # TGroup is numeric ticket with last digit removed - creates another grouping
    dataset['TicketGroup'] = dataset['Ticket'].apply(lambda x: str(int(x.split(' ')[-1])//10))
    dataset['TicketGroup'] = pd.to_numeric(dataset['TicketGroup'], downcast='integer', errors='coerce')

data = [train_data, test_data]

for dataset in data:
    dataset['Companions'] = dataset['Ticket'].duplicated(keep=False).astype(int) * dataset.groupby('Ticket')['Ticket'].transform('count') -1 
    dataset.loc[dataset['Companions'] == -1, 'Companions'] = 0
    # same as companions +1
    dataset['Ticket_Frequency'] = dataset.groupby('Ticket')['Ticket'].transform('count')
    # dup_tickets = train_data.groupby('Ticket').size()
    # train_data['DupTickets'] = train_data['Ticket'].map(dup_tickets)

In [4]:
# Name
############################################################################################################################################
"""
- Title
- Surname groupings
"""

def extract_surname(data):    
    families = []
    for i in range(len(data)):        
        name = data.iloc[i]
        if '(' in name:
            name_no_bracket = name.split('(')[0] 
        else:
            name_no_bracket = name
        family = name_no_bracket.split(',')[0]
        title = name_no_bracket.split(',')[1].strip().split(' ')[0]
        
        for c in string.punctuation:
            family = family.replace(c, '').strip()
            
        families.append(family)           
    return families
    
data = [train_data, test_data]
for dataset in data:
    dataset['Family'] = extract_surname(dataset['Name'])

def get_title(name):
    title_search=re.search(' ([A-Za-z]+)\.', name)
    if title_search:
        return title_search.group(1)
    return ""

Title_Dictionary={"Capt": "Officer", "Col":"Officer", "Major":"Officer",
                  "Jonkheer":"Royal","Don":"Royal","Sir":"Royal", "Lady":"Royal",
                  "Dr":"Officer","Rev":"Officer",
                  "Countess":"Royal", "Dona":"Royal",
                  "Mme":"Mrs", "Ms":"Mrs", "Mrs":"Mrs",
                  "Mlle":"Miss",   "Miss":"Miss",
                  "Mr":"Mr",
                  "Master":"Master"}

def titlemap(x):
    return Title_Dictionary[x]
    
data = [train_data, test_data]
for dataset in data:
    dataset["Title"]=dataset["Name"].apply(get_title)
    dataset["Title"]=dataset["Title"].apply(titlemap)
   
train_data=train_data.drop(["Name"], axis=1)
test_data=test_data.drop(["Name"], axis=1)
"""
df_all['Title'] = df_all['Name'].str.split(', ', expand=True)[1].str.split('.', expand=True)[0]
df_all['Is_Married'] = 0
df_all['Is_Married'].loc[df_all['Title'] == 'Mrs'] = 1
"""

"\ndf_all['Title'] = df_all['Name'].str.split(', ', expand=True)[1].str.split('.', expand=True)[0]\ndf_all['Is_Married'] = 0\ndf_all['Is_Married'].loc[df_all['Title'] == 'Mrs'] = 1\n"

In [45]:
train_data.Prefix.value_counts()

Prefix
NoPrefix    661
Prefixed    230
Name: count, dtype: int64

In [54]:
train_data.info()
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   PassengerId       891 non-null    int64   
 1   Survived          891 non-null    int64   
 2   Pclass            891 non-null    int64   
 3   Sex               891 non-null    object  
 4   Age               891 non-null    float64 
 5   SibSp             891 non-null    int64   
 6   Parch             891 non-null    int64   
 7   Ticket            891 non-null    object  
 8   Fare              891 non-null    float64 
 9   Embarked          891 non-null    object  
 10  Cabin_count       891 non-null    int64   
 11  Deck              891 non-null    object  
 12  Prefix            891 non-null    object  
 13  TicketNumber      891 non-null    int64   
 14  TNlength          891 non-null    int64   
 15  LeadingDigit      891 non-null    int64   
 16  TicketGroup       891 non-

# Missing Values
 - Age: subgroup medians, others have sampled uniform [20,40] or normal distr  
 - Embarked: researched true value
 - Fare: subgroup median  
 - Port: from shared ticket

In [5]:
# Filling the missing values in Age with the medians of Sex and Pclass groups
data = [train_data, test_data]
for dataset in data:
    grouped = dataset.groupby(['Sex', 'Pclass'])['Age'].transform('median')
    dataset['Age']=dataset['Age'].fillna(grouped)
    #dataset['Age']=dataset['Age'].fillna(grouped)
    dataset['Embarked'] = dataset['Embarked'].fillna('S') # commonly known datapoint

    med_fare = dataset.groupby(['Pclass', 'Parch', 'SibSp']).Fare.median()[3][0][0]
# Filling the missing value in Fare with the median Fare of 3rd class alone passenger
    dataset['Fare'] = dataset['Fare'].fillna(med_fare)

"""
from sklearn.impute import KNNImputer, SimpleImputer

imputer=SimpleImputer(strategy='constant',#'mean',
                      fill_value=-999,add_indicator=True).set_output(transform="pandas")
#imputer=KNNImputer(add_indicator=True).set_output(transform="pandas")

train_data=imputer.fit_transform(train_data)
test_data=imputer.fit_transform(test_data)
"""

'\nfrom sklearn.impute import KNNImputer, SimpleImputer\n\nimputer=SimpleImputer(strategy=\'constant\',#\'mean\',\n                      fill_value=-999,add_indicator=True).set_output(transform="pandas")\n#imputer=KNNImputer(add_indicator=True).set_output(transform="pandas")\n\ntrain_data=imputer.fit_transform(train_data)\ntest_data=imputer.fit_transform(test_data)\n'

# Binning

In [6]:
data = [train_data, test_data]
for dataset in data:
    dataset['AgeBand']=pd.cut(dataset['Age'], bins=[0,5,12,18,35,60,np.inf], labels=[0,1,2,3,4,5])
    #dataset['Age'] = pd.qcut(dataset['Age'], 10)
    dataset['AgeBand']=dataset['AgeBand'].astype(int)
    dataset['FareBand']=pd.qcut(dataset['Fare'],q=13, labels=[0,1,2,3,4,5,6,7,8,9,10,11,12])
    dataset['FareBand']=dataset['FareBand'].astype(int)

# Engineering

In [7]:
data = [train_data, test_data]
for dataset in data:
    dataset['Family_Size']=dataset['SibSp']+dataset['Parch']+1
    #dataset['ParchXPclass']=dataset.Parch*dataset.Pclass
    #dataset['SibSpXPclass']=dataset.SibSp*dataset.Pclass
    dataset['AgeXPclass']=dataset.Age*dataset.Pclass
    dataset['AgeXFare']=dataset.Age*dataset.Fare
    
    Family_size=dataset.Parch+dataset.SibSp
    dataset['Alone']=Family_size==0

    family_map = {1: 'Alone', 2: 'Small', 3: 'Small', 4: 'Small', 5: 'Medium', 6: 'Medium', 7: 'Large', 8: 'Large', 11: 'Large'}
    dataset['Family_Size_Grouped'] = dataset['Family_Size'].map(family_map)

In [8]:
train_data.info()
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 28 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PassengerId          891 non-null    int64  
 1   Survived             891 non-null    int64  
 2   Pclass               891 non-null    int64  
 3   Sex                  891 non-null    object 
 4   Age                  891 non-null    float64
 5   SibSp                891 non-null    int64  
 6   Parch                891 non-null    int64  
 7   Ticket               891 non-null    object 
 8   Fare                 891 non-null    float64
 9   Embarked             891 non-null    object 
 10  Cabin_count          891 non-null    int64  
 11  Deck                 891 non-null    object 
 12  Prefix               891 non-null    object 
 13  TicketNumber         891 non-null    int64  
 14  TNlength             891 non-null    int64  
 15  LeadingDigit         891 non-null    int

# Mutual Information

In [10]:
X = train_data.copy()
#X=X.drop(['Cabin','Port'],axis=1)
y = X.pop("Survived")
for colname in X.select_dtypes("object"):
    X[colname], _ = X[colname].factorize()
    
X2 = test_data.copy()
#X2=X2.drop(['Cabin','Port'],axis=1)
for colname in X2.select_dtypes("object"):
    X2[colname], _ = X2[colname].factorize()

# All discrete features should now have integer dtypes (double-check this before using MI!)
discrete_features = X.dtypes == int
discrete_features

from sklearn.feature_selection import mutual_info_classif

def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_classif(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_scores(X, y, discrete_features)
mi_scores

PassengerId            0.665912
Ticket                 0.572496
TicketNumber           0.570940
Family                 0.548683
Title                  0.169489
Sex                    0.150870
Fare                   0.137645
TicketGroup            0.114302
AgeXPclass             0.111545
AgeXFare               0.082590
FareBand               0.067594
Pclass                 0.058107
Companions             0.057587
Ticket_Frequency       0.057587
LeadingDigit           0.053648
Deck                   0.051222
Cabin_count            0.051054
Family_Size            0.047781
Alone                  0.047016
Family_Size_Grouped    0.042141
Age                    0.034382
TNlength               0.026846
SibSp                  0.023197
Parch                  0.016366
Embarked               0.014233
AgeBand                0.012971
Prefix                 0.000001
Name: MI Scores, dtype: float64

# Pre Simple Fit

In [ ]:
"""
train_data2=pd.read_csv("/kaggle/input/titanic/train.csv")
df = train_data2.drop(['Ticket','Cabin'], axis=1)

df2=df[['Pclass','Sex','Age','SibSp','Parch','Embarked','Survived']]
df2['Pclass']=df2['Pclass'].astype("category")

df2 = pd.concat([pd.get_dummies(df2[['Pclass','Sex','Embarked']], drop_first=True), 
                  df2[['Survived','Age','SibSp','Parch']]],axis=1)

test_data2=pd.read_csv("/kaggle/input/titanic/test.csv")
grouped = test_data2.groupby(['Sex', 'Pclass'])['Age'].transform('median')
test_data2['Age']=test_data2['Age'].fillna(grouped)

df3=test_data2[['Pclass','Sex','Age','SibSp','Parch','Embarked']]
df3['Pclass']=df3['Pclass'].astype("category")
df3 = pd.concat([pd.get_dummies(df3[['Pclass','Sex','Embarked']], drop_first=True), 
                  df3[['Age','SibSp','Parch']]],axis=1)
"""

In [39]:
from sklearn.model_selection import train_test_split
import sklearn.linear_model as lm
import sklearn.ensemble as en
import sklearn.gaussian_process as gp
import sklearn.svm as svm
import sklearn.discriminant_analysis as da 
import sklearn.naive_bayes as nb
import lightgbm as lgb
import xgboost as xgb
import catboost 

X_train, X_test,y_train, y_test=train_test_split(X,y, test_size=0.3, random_state=21)

#instantiate and fit our model
# Best = 
#results_rf = da.LinearDiscriminantAnalysis().fit(X_train, y_train) #2
results_rf = catboost.CatBoostClassifier(min_data_in_leaf=5,n_estimators=300,
                                         verbose=0,max_depth= 10,l2_leaf_reg=5,
                                        learning_rate=0.01).fit(X_train, y_train) #1
#results_rf = lgb.LGBMClassifier(verbose=-1).fit(X_train, y_train) #3
#results_rf = xgb.XGBClassifier().fit(X_train, y_train) #4
#results_rf=lm.LogisticRegression().fit(X_train, y_train)
# interesting......LDA is better than XGBoost?

# The Rest
#da.QuadraticDiscriminantAnalysis().fit(X_train, y_train)
#gp.GaussianProcessClassifier().fit(X_train, y_train)
#nb.GaussianNB().fit(X_train, y_train)
#svm.SVC().fit(X_train, y_train)
#en.HistGradientBoostingClassifier().fit(X_train, y_train)

# Score the results
score = results_rf.score(X_test, y_test)
print("Accuracy of model on the data was: {0}".format(score))

Accuracy of model on the data was: 0.8582089552238806


In [ ]:
submission = pd.DataFrame()
submission['PassengerId']=test_data['PassengerId']
submission['Survived'] = results_rf.predict(X2).tolist()
submission.info()
submission.to_csv("submission.csv", index=False)
# test score 0.7535

# Exploratory Data Analysis
Numerical: 'Age', 'SibSp','Parch', 'Fare', 'TNlength', Cabin_count, Companions  
Ordinal: Pclass, 'TicketNumber', 'LeadingDigit', 'TicketGroup',   
Categorical: 'Sex', 'Family', 'Title','Embarked','Prefix', Deck  

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#df_plot = train_data.groupby(['Survived','Family']).size().reset_index().pivot(columns='Survived', index='Family', values=0)
#df_plot.plot(kind='bar', stacked=True)
def plot_stack(column_1, column_2):
 plot_stck=pd.crosstab(index=column_1, columns=column_2)
 plot_stck.plot(kind='barh', figsize=(8,8), stacked=True)
 return
plot_stack(train_data['Family'],train_data['Survived'])
#train_data.head()
#sns.countplot(x='Family',hue='Survived',data=train_data)

In [ ]:
fig, axs = plt.subplots(nrows=2,sharex=True, figsize=(10, 8))
sns.violinplot(y='Fare',x='LeadingDigit', hue='Survived',data=train_data, kde=True,                split=True, ax=axs[0])
sns.violinplot(y='Age',x='LeadingDigit', hue='Survived',data=train_data, kde=True,                split=True, ax=axs[1])

In [ ]:
sns.countplot(x='Cabin_count', hue='Survived', data=train_data)

In [ ]:
fig, axs = plt.subplots(nrows=3,sharex=True, figsize=(10, 8))
sns.countplot(x="Parch",hue="Survived", data=train_data, ax=axs[0]
             #bins=40, element="step", stat="density", common_norm=False#kde=True, #multiple="stack"
           )
sns.countplot(x="SibSp",hue="Survived", data=train_data, ax=axs[1])
sns.countplot(x="Family_Size",hue="Survived",data=train_data, ax=axs[2])

In [ ]:
sns.histplot(x='Family_Size_Grouped', hue='Survived',data=train_data, kde=True)

In [ ]:
fig, axs = plt.subplots(ncols=5,nrows=4, figsize=(15, 20))
sns.scatterplot(y='LeadingDigit', x='SibSp', hue='Survived', data=train_data, alpha=0.2, ax=axs[0,0])
sns.scatterplot(y='LeadingDigit', x='Parch', hue='Survived', data=train_data, alpha=0.2, ax=axs[0,1])
sns.scatterplot(y='LeadingDigit', x='TNlength', hue='Survived', data=train_data, alpha=0.2, ax=axs[0,2])
sns.scatterplot(y='LeadingDigit', x='Cabin_count', hue='Survived', data=train_data, alpha=0.2, ax=axs[0,3])
sns.scatterplot(y='LeadingDigit', x='TicketGroup', hue='Survived', data=train_data, alpha=0.2, ax=axs[0,4])
sns.scatterplot(y='Pclass', x='SibSp', hue='Survived', data=train_data, alpha=0.2, ax=axs[1,0])
sns.scatterplot(y='Pclass', x='Parch', hue='Survived', data=train_data, alpha=0.2, ax=axs[1,1])
sns.scatterplot(y='Pclass', x='TNlength', hue='Survived', data=train_data, alpha=0.2, ax=axs[1,2])
sns.scatterplot(y='Pclass', x='Cabin_count', hue='Survived', data=train_data, alpha=0.2, ax=axs[1,3])
sns.scatterplot(y='Pclass', x='TicketGroup', hue='Survived', data=train_data, alpha=0.2, ax=axs[1,4])
sns.scatterplot(y='TicketNumber', x='SibSp', hue='Survived', data=train_data, alpha=0.2, ax=axs[2,0])
sns.scatterplot(y='TicketNumber', x='Parch', hue='Survived', data=train_data, alpha=0.2, ax=axs[2,1])
sns.scatterplot(y='TicketNumber', x='TNlength', hue='Survived', data=train_data, alpha=0.2, ax=axs[2,2])
sns.scatterplot(y='TicketNumber', x='Cabin_count', hue='Survived', data=train_data, alpha=0.2, ax=axs[2,3])
sns.scatterplot(y='TicketNumber', x='TicketGroup', hue='Survived', data=train_data, alpha=0.2, ax=axs[2,4])
sns.scatterplot(y='TicketGroup', x='SibSp', hue='Survived', data=train_data, alpha=0.2, ax=axs[3,0])
sns.scatterplot(y='TicketGroup', x='Parch', hue='Survived', data=train_data, alpha=0.2, ax=axs[3,1])
sns.scatterplot(y='TicketGroup', x='TNlength', hue='Survived', data=train_data, alpha=0.2, ax=axs[3,2])
sns.scatterplot(y='TicketGroup', x='Cabin_count', hue='Survived', data=train_data, alpha=0.2, ax=axs[3,3])
sns.scatterplot(y='TicketGroup', x='TicketGroup', hue='Survived', data=train_data, alpha=0.2, ax=axs[3,4])

In [ ]:
sns.relplot(x="Age",y='Fare',col='Sex', row='Deck',hue="Survived",data=train_data)

In [ ]:
sns.violinplot(x ="Sex", y ="Age", hue ="Survived",  data = train_data, split = True) 

In [ ]:
"""
# Experimenting with Bin Number
train_data['Fare_Range'] = pd.qcut(train_data['Fare'], 30) 
train_data['Age_Range'] = pd.qcut(train_data['Age'], 12) 
cols=['Fare_Range','Age_Range']
X = train_data[cols]
for colname in X.select_dtypes("category"):
    X[colname], _ = X[colname].factorize()
y = train_data["Survived"]
discrete_features = X.dtypes == int

mi_scores = mutual_info_classif(X, y,discrete_features=discrete_features)

train_data=train_data.drop('Fare_Range', axis=1)
train_data=train_data.drop('Age_Range', axis=1)
mi_scores
"""

In [ ]:
sns.relplot(x="Age",y='Fare',col='Embarked', row='Pclass',hue="Survived",data=train_data)

In [ ]:
sns.histplot(x="Fare",y='Parch',hue="Survived", bins=50,data=train_data)#.set_xscale('log')
sns.histplot(x="SibSp",y='Parch',hue="Survived",data=train_data).set_yscale('log')

In [ ]:
sns.histplot(x="Parch",hue="Survived",bins =5,data=train_data).set_yscale('log')
sns.histplot(x="Pclass",hue="Survived",bins=3,data=train_data)#.set_xscale('log')

## 2. Categorical
name, sex, embarked, cabin

In [ ]:
sns.countplot(x="Sex",hue="Survived", data=train_data)
sns.countplot(x="Embarked",hue="Survived", data=train_data)
sns.countplot(x='Companions', hue='Survived', data=train_data)

# Processing  
## Ideas  
https://medium.com/@praoiticica/titanic-data-cleaning-and-feature-engineering-9f122752097f  
https://www.kaggle.com/code/gunesevitan/titanic-advanced-feature-engineering-tutorial/notebook   

## Creating Features
 - Math transforms  
 - Counts (across cols): X["Components"] = X[components].gt(0).sum(axis=1)  
 - Building up / breaking down  
 - Grouped  
 - clustering  
 - target encoding  
 - PCA  
 - 

In [ ]:
"""
def concat_df(train_data, test_data):
    # Returns a concatenated df of training and test set
    return pd.concat([train_data, test_data], sort=True).reset_index(drop=True)

def divide_df(all_data):
    # Returns divided dfs of training and test set
    return all_data.loc[:890], all_data.loc[891:].drop(['Survived'], axis=1)

df_train = pd.read_csv("/kaggle/input/titanic/train.csv")
df_test = pd.read_csv("/kaggle/input/titanic/test.csv")
df_all = concat_df(df_train, df_test)

df_train.name = 'Training Set'
df_test.name = 'Test Set'
df_all.name = 'All Set' 

dfs = [df_train, df_test]

print('Number of Training Examples = {}'.format(df_train.shape[0]))
print('Number of Test Examples = {}\n'.format(df_test.shape[0]))
print('Training X Shape = {}'.format(df_train.shape))
print('Training y Shape = {}\n'.format(df_train['Survived'].shape[0]))
print('Test X Shape = {}'.format(df_test.shape))
print('Test y Shape = {}\n'.format(df_test.shape[0]))
print(df_train.columns)
print(df_test.columns)
"""

# Engineering

In [ ]:
"""
def isRare(title):
    if title == "Mr" or title == "Mrs" or title == "Master" or title == "Miss":
        return 0
    return 1
data = [train_data, test_data]

for dataset in data:
    dataset["TitleRarity"] = dataset["Title"].apply(isRare)

sns.countplot(x='TitleRarity', hue='Survived', data=train_data)
"""

# Target Encoding

In [58]:
# Creating a list of families and tickets that are occuring in both training and test set
non_unique_families = [x for x in train_data['Family'].unique() if x in test_data['Family'].unique()]
non_unique_tickets = [x for x in train_data['Ticket'].unique() if x in test_data['Ticket'].unique()]

df_family_survival_rate = train_data.groupby('Family')[['Survived', 'Family_Size']].median()
df_ticket_survival_rate = train_data.groupby('Ticket')[['Survived', 'Ticket_Frequency']].median()

family_rates = {}
ticket_rates = {}

for i in range(len(df_family_survival_rate)):
    # Checking a family exists in both training and test set, and has members more than 1
    if df_family_survival_rate.index[i] in non_unique_families and df_family_survival_rate.iloc[i, 1] > 1:
        family_rates[df_family_survival_rate.index[i]] = df_family_survival_rate.iloc[i, 0]

for i in range(len(df_ticket_survival_rate)):
    # Checking a ticket exists in both training and test set, and has members more than 1
    if df_ticket_survival_rate.index[i] in non_unique_tickets and df_ticket_survival_rate.iloc[i, 1] > 1:
        ticket_rates[df_ticket_survival_rate.index[i]] = df_ticket_survival_rate.iloc[i, 0]

print (df_family_survival_rate.head(2))
print ("\n", list(family_rates.items())[:2])      

print ("\n\n", df_ticket_survival_rate.head(2))
print ("\n", list(ticket_rates.items())[:2])       

        Survived  Family_Size
Family                       
Abbing       0.0          1.0
Abbott       0.5          3.0

 [('Abbott', 0.5), ('Aks', 1.0)]


         Survived  Ticket_Frequency
Ticket                            
110152       1.0               3.0
110413       1.0               3.0

 [('113781', 0.5), ('11767', 1.0)]


In [59]:
mean_survival_rate = np.mean(train_data['Survived'])

train_family_survival_rate = []
train_family_survival_rate_NA = []
test_family_survival_rate = []
test_family_survival_rate_NA = []

for i in range(len(train_data)):
    if train_data['Family'][i] in family_rates:
        train_family_survival_rate.append(family_rates[train_data['Family'][i]])
        train_family_survival_rate_NA.append(1)
    else:
        train_family_survival_rate.append(mean_survival_rate)
        train_family_survival_rate_NA.append(0)
        
for i in range(len(test_data)):
    if test_data['Family'].iloc[i] in family_rates:
        test_family_survival_rate.append(family_rates[test_data['Family'].iloc[i]])
        test_family_survival_rate_NA.append(1)
    else:
        test_family_survival_rate.append(mean_survival_rate)
        test_family_survival_rate_NA.append(0)
        
train_data['Family_Survival_Rate'] = train_family_survival_rate
train_data['Family_Survival_Rate_NA'] = train_family_survival_rate_NA
test_data['Family_Survival_Rate'] = test_family_survival_rate
test_data['Family_Survival_Rate_NA'] = test_family_survival_rate_NA

train_ticket_survival_rate = []
train_ticket_survival_rate_NA = []
test_ticket_survival_rate = []
test_ticket_survival_rate_NA = []

for i in range(len(train_data)):
    if train_data['Ticket'][i] in ticket_rates:
        train_ticket_survival_rate.append(ticket_rates[train_data['Ticket'][i]])
        train_ticket_survival_rate_NA.append(1)
    else:
        train_ticket_survival_rate.append(mean_survival_rate)
        train_ticket_survival_rate_NA.append(0)
        
for i in range(len(test_data)):
    if test_data['Ticket'].iloc[i] in ticket_rates:
        test_ticket_survival_rate.append(ticket_rates[test_data['Ticket'].iloc[i]])
        test_ticket_survival_rate_NA.append(1)
    else:
        test_ticket_survival_rate.append(mean_survival_rate)
        test_ticket_survival_rate_NA.append(0)
        
train_data['Ticket_Survival_Rate'] = train_ticket_survival_rate
train_data['Ticket_Survival_Rate_NA'] = train_ticket_survival_rate_NA
test_data['Ticket_Survival_Rate'] = test_ticket_survival_rate
test_data['Ticket_Survival_Rate_NA'] = test_ticket_survival_rate_NA

#
for df in [train_data, test_data]:
    df['Survival_Rate'] = (df['Ticket_Survival_Rate'] + df['Family_Survival_Rate']) / 2
    df['Survival_Rate_NA'] = (df['Ticket_Survival_Rate_NA'] + df['Family_Survival_Rate_NA']) / 2 

# Transformation

In [ ]:
train_data.head().T

In [60]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# Label and One Hot Encoding
non_numeric_features = ['Embarked', 'Sex', 'Deck', 'Title', 'Family_Size_Grouped', 'AgeBand', 'FareBand','Prefix']

for df in [train_data, test_data]:
    for feature in non_numeric_features:        
        df[feature] = LabelEncoder().fit_transform(df[feature])

cat_features = ['Pclass', 'Sex', 'Deck', 'Embarked', 'Title', 'Family_Size_Grouped','Prefix']

print (f"Before one hot encoding cat features: {train_data.shape}") 
train_data = pd.get_dummies(data=train_data, columns=cat_features, drop_first=True)
print (f"After one hot encoding cat features: {train_data.shape}") 

print (f"Before one hot encoding cat features: {test_data.shape}") 
test_data = pd.get_dummies(data=test_data, columns=cat_features, drop_first=True)
print (f"After one hot encoding cat features: {test_data.shape}") 

Before one hot encoding cat features: (891, 34)
After one hot encoding cat features: (891, 44)
Before one hot encoding cat features: (418, 33)
After one hot encoding cat features: (418, 43)


In [30]:
train_data.info()
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 72 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   PassengerId              891 non-null    int64  
 1   Survived                 891 non-null    int64  
 2   Age                      891 non-null    float64
 3   SibSp                    891 non-null    int64  
 4   Parch                    891 non-null    int64  
 5   Ticket                   891 non-null    object 
 6   Fare                     891 non-null    float64
 7   Cabin_count              891 non-null    int64  
 8   TicketNumber             891 non-null    int64  
 9   TNlength                 891 non-null    int64  
 10  LeadingDigit             891 non-null    int64  
 11  TicketGroup              891 non-null    int32  
 12  Companions               891 non-null    int64  
 13  Ticket_Frequency         891 non-null    int64  
 14  AgeBand                  8

In [61]:
drop_cols = (
    # 1. cat features (no need to need dropped as dummy_encoding takes care of them)
    #['Pclass', 'Sex', 'Deck', 'Embarked', 'Title', 'Family_Size_Grouped']
    # 2. objects
    [#'Name',
        'Ticket', 'Family'] 
    # 3. other cols
    + ['Family_Size', 'Parch', 'PassengerId', 'SibSp', 'Ticket_Survival_Rate', 'Family_Survival_Rate', 'Ticket_Survival_Rate_NA', 'Family_Survival_Rate_NA'] 
    # 4. target col
    + ['Survived'])
drop_cols2 = (
    # 1. cat features (no need to need dropped as dummy_encoding takes care of them)
    #['Pclass', 'Sex', 'Deck', 'Embarked', 'Title', 'Family_Size_Grouped']
    # 2. objects
    [#'Name',
        'Ticket', 'Family'] 
    # 3. other cols
    + ['Family_Size', 'Parch', 'PassengerId', 'SibSp', 'Ticket_Survival_Rate', 'Family_Survival_Rate', 'Ticket_Survival_Rate_NA', 'Family_Survival_Rate_NA'] 
    # 4. target col
    )

In [62]:
from sklearn.preprocessing import StandardScaler

X = StandardScaler().fit_transform(train_data.drop(columns=drop_cols))
y = train_data['Survived'].values
X2 = StandardScaler().fit_transform(test_data.drop(columns=drop_cols2))
"""
# Manual Scaling
data = [train_data, test_data]
for dataset in data:
    numeric_columns=['Age','SibSp','Parch','Fare','DupTickets','TNlen','LeadingDigit']
    mu=dataset[numeric_columns].mean(axis=0)
    std=dataset[numeric_columns].std(axis=0)
    dataset[numeric_columns]=(dataset[numeric_columns]-mu/std)
"""

"\n# Manual Scaling\ndata = [train_data, test_data]\nfor dataset in data:\n    numeric_columns=['Age','SibSp','Parch','Fare','DupTickets','TNlen','LeadingDigit']\n    mu=dataset[numeric_columns].mean(axis=0)\n    std=dataset[numeric_columns].std(axis=0)\n    dataset[numeric_columns]=(dataset[numeric_columns]-mu/std)\n"

In [72]:
y_train

array([0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1,

# Modeling

In [ ]:
#feature selection
import catboost
from sklearn.feature_selection import RFECV
estimator=catboost.CatBoostClassifier(verbose=0)
selector = RFECV(estimator, step=1, cv=5)
selector = selector.fit(X_train, y_train)
selector.ranking_

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import sklearn.linear_model as lm
import sklearn.ensemble as en
import sklearn.gaussian_process as gp
import sklearn.svm as svm
import sklearn.discriminant_analysis as da 
import sklearn.naive_bayes as nb
import lightgbm as lgb
import xgboost as xgb
import catboost 
import sklearn

X_train, X_test,y_train, y_test=train_test_split(X,y, test_size=0.2, random_state=21)

model=lm.LogisticRegression()
#catboost.CatBoostClassifier(verbose=0)
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
accuracy =accuracy_score(y_test, y_pred)
print(f'Log Reg Accuracy:{accuracy:.4f}')

model=catboost.CatBoostClassifier(verbose=0) #depth=6, iterations=500, learning_rate=0.01,l2_leaf_reg=5, 
   # 'depth':[3,1,2,6,4,5,7,8,9,10],
    #          'iterations':[250,100,500,1000],
    #          'learning_rate':[0.03,0.001,0.01,0.1,0.2,0.3],
     #         'l2_leaf_reg':[3,1,5,10,100],
      #        'border_count':[32,5,10,20,50,100,200],
       #       'bagging_temperature':[0.03,0.09,0.25,0.75],
        #      'random_strength':[0.2,0.5,0.8],
         #'max_ctr_complexity':[1,2,3,4,5] 
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
accuracy =accuracy_score(y_test, y_pred)
print(f'catboost Accuracy:{accuracy:.4f}')

model=xgb.XGBClassifier()
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
accuracy =accuracy_score(y_test, y_pred)
print(f'XGB Accuracy:{accuracy:.4f}')

model=svm.SVC()
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
accuracy =accuracy_score(y_test, y_pred)
print(f'SVM Accuracy:{accuracy:.4f}')

model=da.LinearDiscriminantAnalysis()
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
accuracy =accuracy_score(y_test, y_pred)
print(f'LDA Accuracy:{accuracy:.4f}')

model=nb.GaussianNB()
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
accuracy =accuracy_score(y_test, y_pred)
print(f'NB Accuracy:{accuracy:.4f}')

model=lgb.LGBMClassifier(verbose=-1)
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
accuracy =accuracy_score(y_test, y_pred)
print(f'LGBM Accuracy:{accuracy:.4f}')

Log Reg Accuracy:0.8436


In [ ]:
# Submission
submission = pd.DataFrame()
submission['PassengerId']=pd.to_numeric(test_data['PassengerId'],downcast='integer')
submission['Survived'] = pd.to_numeric(lgb_classifier.predict(X_test).tolist(),downcast='integer')
submission.head()
submission.to_csv("submission.csv", index=False)

In [ ]:
from sklearn import linear_model, svm
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score

# Logistic Regression
#model=linear_model.LogisticRegression(penalty='elasticnet', dual= False, solver='saga', tol=0.001, C=100, max_iter=10000, l1_ratio=0.4)

# LGBM
model=catboost.CatBoostClassifier(verbose=0)

#=lgb.LGBMClassifier(objective='binary', max_depth=6, learning_rate=0.007, n_estimators=100, verbose=-1,min_data_in_leaf=25, num_iterations=1000, num_leaves=100, max_bin=10)

# SVM
#model=svm.SVC(C=100, kernel='poly', degree=5, gamma='scale', coef0=0.0, shrinking=True, tol=0.0001, class_weight=None, verbose=False, max_iter=-1, decision_function_shape='ovo', 
#break_ties=False)
# LDA
#model=sklearn.discriminant_analysis.LinearDiscriminantAnalysis()
# QDA
#model=sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis()
# Gaussian Process
#model=
# Naive Bayes
#model=sklearn.naive_bayes.GaussianNB()
# Voting Classifier
#model=sklearn.ensemble.VotingClassifier(
# AdaBoost
#model =sklearn.ensemble.AdaBoostClassifier(
# Supervised Nearest Neighbors
#model=sklearn.neighbors.KNeighborsClassifier() 

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

#log_reg.fit(X_train, y_train)
#y_pred=log_reg.predict(X_test)

#accuracy =accuracy_score(y_test, y_pred)
#print(f'Accuracy:{accuracy:.2f}')

In [ ]:
# Calibration
from sklearn.calibration import CalibratedClassifierCV
X_train, X_calib, y_train, y_calib = train_test_split(X, y, random_state=42)
base_clf = linear_model.LogisticRegression()
base_clf.fit(X_train, y_train)

from sklearn.frozen import FrozenEstimator
calibrated_clf = CalibratedClassifierCV(FrozenEstimator(base_clf))
calibrated_clf.fit(X_calib, y_calib)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(calibrated_clf, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

# Simple Ensemble Model

In [157]:
from sklearn import svm

X_train, X_test,y_train, y_test=train_test_split(X,y, test_size=0.2, random_state=21)

lgbm = lgb.LGBMClassifier(objective = 'binary', n_estimators=500, verbose=-1).fit(X_train, y_train)
svmc = svm.SVC(C = 5, probability = True).fit(X_train, y_train)
lr=lm.LogisticRegression(penalty='elasticnet', dual= False, solver='saga', tol=0.001, C=100, max_iter=10000, l1_ratio=0.4).fit(X_train, y_train)
lda=da.LinearDiscriminantAnalysis().fit(X_train, y_train)
cb=catboost.CatBoostClassifier(iterations=500,learning_rate=0.1,depth=6,l2_leaf_reg=3,cat_features=[],eval_metric='Accuracy',verbose=0).fit(X_train, y_train)
nbayes=nb.GaussianNB().fit(X_train, y_train)
lda=da.LinearDiscriminantAnalysis(n_components=1,solver='svd').fit(X_train, y_train)
xg=xgb.XGBClassifier().fit(X_train, y_train)

# Predict
lgbm_preds = lgbm.predict_proba(X_test).transpose()[1]
lr_preds = lr.predict_proba(X_test).transpose()[1]

svm_preds = svmc.predict_proba(X_test).transpose()[1]
lda_preds = lda.predict_proba(X_test).transpose()[1]
cb_preds = cb.predict_proba(X_test).transpose()[1]
nb_preds = nbayes.predict_proba(X_test).transpose()[1]
lda_preds = lda.predict_proba(X_test).transpose()[1]
xgb_preds= xg.predict_proba(X_test).transpose()[1]

#Assign different weightages to the classifiers
ensemble_preds = lgbm_preds*0.3 + lr_preds*0.7
ensemble_preds2 = lda_preds*0.5 + cb_preds*0.5

for x in range(len(ensemble_preds)):
    if ensemble_preds[x] >= 0.5:
        ensemble_preds[x] = 1
    else:
        ensemble_preds[x] = 0

results  = ensemble_preds.astype(int)

accuracy =accuracy_score(y_test, results)
print(f'Ens1 Accuracy:{accuracy:.4f}')

for x in range(len(ensemble_preds2)):
    if ensemble_preds2[x] >= 0.5:
        ensemble_preds2[x] = 1
    else:
        ensemble_preds2[x] = 0

results  = ensemble_preds2.astype(int)

accuracy =accuracy_score(y_test, results)
print(f'Ens2 Accuracy:{accuracy:.4f}')

Ens1 Accuracy:0.8827
Ens2 Accuracy:0.8771


# Submission

In [152]:
lgbm_preds = lgbm.predict_proba(X2).transpose()[1]
lr_preds = lr.predict_proba(X2).transpose()[1]
cb_preds = cb.predict_proba(X2).transpose()[1]
lda_preds = lda.predict_proba(X2).transpose()[1]
ensemble_preds = cb_preds*0.5 + lda_preds*0.5
for x in range(len(ensemble_preds)):
    if ensemble_preds[x] >= 0.5:
        ensemble_preds[x] = 1
    else:
        ensemble_preds[x] = 0

results  = ensemble_preds.astype(int)

submission = pd.DataFrame()
submission['Survived'] = results.tolist()
submission['PassengerId']=test_data['PassengerId']
submission.info()
submission.to_csv("submission.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   Survived     418 non-null    int64
 1   PassengerId  418 non-null    int64
dtypes: int64(2)
memory usage: 6.7 KB


# Ensembled Model

In [ ]:
model=sklearn.ensemble.StackingClassifier()

# Hyperparameter Tuning

In [ ]:
import optuna
from sklearn import linear_model

X_train, X_test,y_train, y_test=train_test_split(X,y, test_size=0.2, random_state=21)

def objective(trial):
    # Suggest hyperparameters
    params = {
        'penalty':'elasticnet',
        'dual': False,
        'solver':'saga',
        'tol': trial.suggest_int("tol", 0.000001,0.001),
        'C':trial.suggest_float("C", 0.0, 10.0),
        'max_iter':trial.suggest_int("max_iter", 10, 500, step=5),
        'l1_ratio': trial.suggest_float("l1_ratio", 0, 1.0, step=0.1)       
    }

    model = linear_model.LogisticRegression(**params)
    scores = cross_val_score(model, X_train, y_train, cv=10, scoring='f1', verbose=0, error_score='raise')
    
    return scores.mean()
    
# Create a study object
study = optuna.create_study(direction="maximize")

# Optimize the objective function
study.optimize(objective, n_trials=20)

print("Best trial:", study.best_trial)
print("Best hyperparameters:", study.best_params)
print("Best value:", study.best_value)

best_params = study.best_params
params = {k: v for k, v in best_params.items()} # Add prefix for set_params function

In [ ]:
# Metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, ConfusionMatrixDisplay

best_model = linear_model.LogisticRegression(**params)

# Initialize and train the best model
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='binary')
recall = recall_score(y_test, y_pred, average='binary')
class_report = classification_report(y_test, y_pred)

# Print the evaluation metrics
print(f'Accuracy: {accuracy:.2f}')

# Processing

In [ ]:
ignore_features=['Survived','PassengerId']
cat_col = [col for col, type in train_data.dtypes.items() if type in ["category", 'object']and col not in ignore_features]
num_col = [col for col, type in train_data.dtypes.items() if type in ['float', 'int64']and col not in ignore_features]
features = num_col + cat_col

X=train_data.drop(ignore_features, axis=1)
X_test=test_data.drop('PassengerId', axis=1)
#train_data['Survived'] = np.where(df['target'] == "target", 1, 0)
y= train_data['Survived'].values

# Data Pipeline

In [ ]:
from typing import List
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn import linear_model
from sklearn.feature_extraction import DictVectorizer
import lightgbm as lgb

# Convert df into a dictionary: df_dict
#df_dict = df.to_dict("records")

# Create the DictVectorizer object: dv
#dv = DictVectorizer(sparse=False)

# Apply dv on df: df_encoded
#df_encoded = dv.fit_transform(df_dict)

def build_pipeline(num_features: List[str], cat_features: List[str]) -> Pipeline:
    """Full pipeline

    This function constructs the whole pipeline for training

    Params:
        config (Dict): Config content from yaml
        num_features (List[str]): List of numeric features
        cat_features (List[str]): List of categorial feature

    Returns:
        Pipeline that contians pre-process, sampling (If specified) and model

    Note:
        * Config assumes we're in the `pipeline` level already
    """
    numeric_transformer = make_pipeline(SimpleImputer(strategy='constant',fill_value=-999),
                                        StandardScaler()) #strategy='mean'

    categorical_transformer = make_pipeline(SimpleImputer(strategy='most_frequent'), #SimpleImputer(strategy='constant', fill_value='missing'),
                                            OneHotEncoder(handle_unknown='ignore')) #, min_frequency=0.05
                                            

    preprocessor = make_column_transformer((numeric_transformer, num_features),(categorical_transformer, cat_features), remainder="passthrough")
    
    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                            ('model', lgb.LGBMClassifier())]) #linear_model.LogisticRegression()
    return pipe

pipeline = build_pipeline(num_features=num_col, cat_features=cat_col)

# Hyperparameter Tuning

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import optuna 
import lightgbm as lgb

X_train, X_test,y_train, y_test=train_test_split(X,y, test_size=0.2, random_state=21)

def objective(trial):
    # Suggest hyperparameters
    params = {
    #Log Reg
      #  'model__penalty':'elasticnet',
      #  'model__dual': False,
      #  'model__solver':'saga',
      #  'model__tol': trial.suggest_int("tol", 0.000001,0.001),
      #  'model__C':trial.suggest_float("C", 0.0, 10.0),
      #  'model__max_iter':trial.suggest_int("max_iter", 10, 500, step=5),
      #  'model__l1_ratio': trial.suggest_float("l1_ratio", 0, 1.0, step=0.1)       
    # LGBM
        'model__boosting_type':'dart',
        #'model__categorical_feature': 'name:Sex',
        #'model__tree_method':'',  # this parameter means using the GPU when training our model to speedup the training process
        #'model__reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
        #'model__reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
        #'model__colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 1.0),
        #'model__subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'model__learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.01),         
        'model__num_iterations': trial.suggest_int('num_iterations', 100, 1000),
        #'model__max_depth': trial.suggest_int('max_depth',3 , 10),
        'model__num_leaves':trial.suggest_int('num_leaves',10,100),
        'model__max_bin': trial.suggest_int('max_bin', 4, 100),
        'model__n_estimators':trial.suggest_int('n_estimators',100,1000)
        #'model__min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
        #'model__min_child_samples': trial.suggest_int('min_child_samples', 1, 300) # for lgbm    
    # Catboost
        
    
    }
    

    model = lgb.LGBMClassifier(**params)
    scores = cross_val_score(model, X_train, y_train, cv=10, scoring='f1', verbose=0, error_score='raise')
    
    return scores.mean()
    
# Create a study object
study = optuna.create_study(direction="maximize")

# Optimize the objective function
study.optimize(objective, n_trials=20)

print("Best trial:", study.best_trial)
print("Best hyperparameters:", study.best_params)
print("Best value:", study.best_value)

best_params = study.best_params
prefix = "model__"
params = {prefix + k: v for k, v in best_params.items()} # Add prefix for set_params function

# Metrics

In [ ]:
## View feature importance of holdout set model
## Evaluation holdout set
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, ConfusionMatrixDisplay

best_model = lgb.LGBMClassifier(**params)

# Initialize and train the best model
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='binary')
recall = recall_score(y_test, y_pred, average='binary')
class_report = classification_report(y_test, y_pred)

# Print the evaluation metrics
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'Classification Report:\n{class_report}')

# Confusion Matrix
import matplotlib.pyplot as plt 
ConfusionMatrixDisplay.from_predictions(y_test, y_pred)
plt.show()

# Basic Feature Importance Visualisation with Single Decision Tree 

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

p2 = pipeline.named_steps.preprocessor
p2.fit(X_train, y_train)
x_transf = p2.transform(X_test)

# Initialize the model
clf = DecisionTreeClassifier(max_depth=2)

# Fit the model to the data
clf.fit(x_transf, y_test)

# Plot the decision tree
#plt.figure(figsize=(20, 10))
tree.plot_tree(clf, filled=True, feature_names=p2.get_feature_names_out(), class_names=['0','1'])
plt.show()

# Formatting for Submission

In [ ]:
test_data.info()

In [ ]:
submission = pd.DataFrame()
submission['Pred_Survived'] = best_model.predict(test_data).tolist()
submission['PassengerId']=test_data['PassengerId']
submission.info()
submission.to_csv("submission.csv", index=False)